# Microestructura y Sistemas de Trading

## Notas de Administracion Activa

In [1]:
# Remover todos los objetos del "Environment"
rm(list = ls())

# los 0s aceptados antes de expresas una cifra en notacion cientifica
options("scipen"=100, "digits"=4)

# Librerias a utilizar
pkg <- c("Quandl","ROI","knitr","openxlsx","kableExtra","PortfolioAnalytics")

# Verificar si estan instaladas y/o cargar librerias
instalados <- pkg %in% installed.packages()
if(length(pkg[!instalados]) > 0) install.packages(pkg[!instalados])
installedpackages <- lapply(pkg, library, character.only=TRUE)

options(knitr.table.format = "html") 

# Cargar el token de QUANDL
Quandl.api_key("dN9QssXxzTxndaqKUQ_i")

Installing packages into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
also installing the dependency ‘PerformanceAnalytics’

Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

ROI.plugin.nlminb: R Optimization Infrastructure
Registered solver plugins: nlminb.
Default solver: auto.
Loading required package: foreach
Loading required package: PerformanceAnalytics

Attaching package: ‘PerformanceAnalytics’

The following object is masked from ‘package:graphics’:

    legend


Attaching package: ‘PortfolioAnalytics’

The following objects are masked from ‘package:ROI’:

    is.constraint, objective



In [30]:
# Funcion para descagar precios
Bajar_Precios <- function(Columns, Tickers, Fecha_In, Fecha_Fn) {
  
  # Funcion para descargar N cantidad de activos desde QUANDL
  # -- Dependencias: QUANDL
  # -- Columns : columnas a incluir : character : c("date", "adj_close", ... )
  # -- Tickers : Tickers o claves de pizarra de los activos : character : "TSLA"
  # -- Fecha_In : Fecha Inicial : character : "2017-01-02"
  # -- Fecha_Fn : Fecha Final : character : "2017-08-02"
  
  # Peticion para descargar precios
  Datos <- Quandl.datatable("WIKI/PRICES", qopts.columns=Columns,
                            ticker=Tickers,
                            date.gte=Fecha_In, date.lte=Fecha_Fn)
  return(Datos)
}

In [31]:
# Tickers de acciones contenidas en ETF-IAK

Datos_ETF <- read.xlsx("IAK_holdings.xlsx", sheet = 1)
tk  <- as.character(na.omit(Datos_ETF[which(Datos_ETF[,1] == "Ticker")+1:length(Datos_ETF[,1]),1]))
cs <- c("date", "adj_close")

In [32]:
# Fecha inicial y fecha final
fs <- c("2016-09-01", "2018-09-01")

# Descargar Precios
Datos <- list()
for(i in 1:length(tk)) {
  Datos[[i]] <- Bajar_Precios(Columns=cs, Ticker=tk[i], Fecha_In=fs[1], Fecha_Fn=fs[2])
}

names(Datos) <- tk

## Datos Completos

In [33]:
longitudes <- c()

for(i in 1:length(Datos)){
  longitudes[i] <- length(Datos[[i]]$date)
}

maximo <- max(longitudes)
completos <- which(longitudes == maximo)

DatosN <- Datos[completos]

In [34]:
# Vector para almacenar columnas de interes
columnas <- c()
nuevos   <- c()

# Funci?n para repetir una funci?n por cada columna del data.frame
Precios <- do.call(cbind, DatosN)

# Crear vector con nombres de columnas de interes = "nombredeactivo.adj_close_r"
for(i in 1:length(tk)){
  nuevos[i] <- paste(tk[i], ".adj_close", sep="")
}

# Extraer 1 renglon para obtener los nombres de las columnas
nombres <- colnames(Precios[1,(names(Precios) %in% nuevos)])

# Elejir una columna Date y las dem?s columnas de rendimientos
Precios <- Precios[,(names(Precios) %in% nuevos)]
row.names(Precios) <- DatosN[[1]]$date

# Reasignar nombres al data.frame
tk_completos <- as.character(tk[completos])
colnames(Precios) <- tk_completos

In [39]:
Historico <- data.frame("Date" = row.names(Precios),
                        "Precio" = Precios[,2], 
                        "R_Precio" = 0, 
                        "R_Activo" = 0,
                        "R_Cuenta" = 0, 
                        "Capital" = 0, "Balance" = 0, "Titulos" = 0,
                        "Titulos_a" = 0,
                        "Operacion" = NA, "Comisiones" = 0, "Comisiones_a" = 0, "Mensaje" = NA)

# *Date*       : Fecha (Proviene desde los precios que bajaron).
# *Precio*     : Precio individual del activo.
# *R_Precio*   : Rendimiento diario del precio (dia a dia).
# *R_Activo*   : Rendimiento acumulado del precio (Cada dia respecto al precio inicial).
# *Capital*    : El dinero no invertido (Equivalente a Efectivo).
# *Balance*    : El valor del portafolio (Precio diario X Titulos).
# *R_Cuenta*   : Balance + Capital (Cada dia respecto al capital inicial).
# *Titulos*    : Acciones que se tienen.
# *Titulos_a*  : Titulos acumulados.
# *Operacion*  : Indicativo de Compra (1), Mantener (0), Venta (-1).
# *Comisiones* : 0.0025 o 0.25% por el valor de la transaccion.
# *Mensaje*    : Un texto que indique alguna decision o indicativo de que ocurrio algo.

Regla0_R <- -0.03  # Considerar una oportunidad de compra en un rendimiento de -3% o menor.
Regla1_I <- 0.20   # Porcentaje de capital para comprar titulos para posicion Inicial.
Regla2_P <- 0.25   # Se utiliza el P% del L capital restante en cada compra.
Regla3_W <- tk_completos # Se realiza la misma estrategia para todos los activos en el portafolio.
Regla4_C <- 0.0025 # Comisiones pagadas por compra.
Regla5_K <- 100000 # Capital Inicial.


In [40]:
# -- Calcular los Titulos de posicion inicial
Historico$Titulos[1] <- (Regla5_K*Regla1_I)%/%Historico$Precio[1]

# -- Se calculan comisiones iniciales
Historico$Comisiones[1] <- Historico$Titulos[1]*Historico$Precio[1]*Regla4_C

# -- Se calculan comisiones acumuladas
Historico$Comisiones_a[1] <- Historico$Comisiones[1]

# -- Calcular el Balance
Historico$Balance[1] <- Historico$Titulos[1]*Historico$Precio[1]

# -- Todo remanente se dejar? registrado en la cuenta de efectivo.
Historico$Capital[1] <- Regla5_K-Historico$Balance[1]-Historico$Comisiones[1]

# -- Iniciamos con una postura de mantener.
Historico$Operacion[1] <- "Posicion Inicial"

# -- El rendimiento de capital en el tiempo 1 es 0
Historico$R_Cuenta[1] <- 0

# -- Mensaje inicial
Historico$Mensaje[1] <- "Inicializacion de cartera"

# -- Calcular R_Precio
Historico$R_Precio <- round(c(0, diff(log(Historico$Precio))),4)

# -- Calcular R_Activo
for(i in 1:length(Historico$Date)){
  Historico$R_Activo[i] <- round((Historico$Precio[i]/Historico$Precio[1])-1,2)
}

### Calculos de todas las cuentas dia por dia

In [41]:

for(i in 2:length(Historico$Date)){

  if(Historico$R_Precio[i] <= Regla0_R){ # Generar Señal
    
    # Establecer capital actual, inicialmente, igual al capital anterior
    Historico$Capital[i] <- Historico$Capital[i-1]
    
    if(Historico$Capital[i] > 0){ # Si hay capital
      
      if(Historico$Capital[i]*Regla2_P > Historico$Precio[i]){ # Si Capital minimo
        
        Historico$Operacion[i] <- "Compra"
        Historico$Titulos[i]   <- (Historico$Capital[i]*Regla2_P)%/%Historico$Precio[i]
        
        compra <- Historico$Precio[i]*Historico$Titulos[i]  
        Historico$Comisiones[i] <- compra*Regla4_C
        
        Historico$Titulos_a[i] <- Historico$Titulos[i-1]+Historico$Titulos[i]

      }
      
    }
    else { # No hubo capital
      
      
    }
    
    
  }
  else { # Sin señal
  }
}


In [42]:
head(Historico)

Date,Precio,R_Precio,R_Activo,R_Cuenta,Capital,Balance,Titulos,Titulos_a,Operacion,Comisiones,Comisiones_a,Mensaje
2018-03-27,53.78,0.0000,0.00,0,79998,19952,371,0,Posicion Inicial,49.88,49.88,Inicializacion de cartera
2018-03-26,54.21,0.0080,0.01,0,0,0,0,0,NA,0.00,0.00,NA
2018-03-23,53.41,-0.0149,-0.01,0,0,0,0,0,NA,0.00,0.00,NA
2018-03-22,54.81,0.0259,0.02,0,0,0,0,0,NA,0.00,0.00,NA
2018-03-21,55.84,0.0186,0.04,0,0,0,0,0,NA,0.00,0.00,NA
2018-03-20,55.63,-0.0038,0.03,0,0,0,0,0,NA,0.00,0.00,NA
